In [2]:
# importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [3]:
# read the csv file
df=pd.read_csv("df_k.csv")
df

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df = pd.read_csv("df_k.csv")
logging.info("CSV file read successfully")

# Initialize the webdriver
driver_k = webdriver.Chrome()

# Initialize lists to store data
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

def scroll_page(driver):
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

# Loop through each row in the DataFrame
for i, r in df.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    logging.info(f"Processing route: {routes}")

    try:
        # Navigate to the link
        driver_k.get(link)
        time.sleep(2)

        # Click on elements to reveal bus details
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Click elements to view buses
        try:
            clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except NoSuchElementException:
            logging.warning("Button not found, continuing to next route")
            continue
        time.sleep(2)

        # Scroll the page
        scroll_page(driver_k)

        # Extract bus details
        bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Bus_names_k.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        Bus_types_k.extend([elem.text for elem in bus_type])
        Start_Time_k.extend([elem.text for elem in start_time])
        End_Time_k.extend([elem.text for elem in end_time])
        Total_Duration_k.extend([elem.text for elem in total_duration])
        Ratings_k.extend([elem.text for elem in rating])
        Prices_k.extend([elem.text for elem in price])
        Seats_Available_k.extend([elem.text for elem in seats])

    except Exception as e:
        logging.error(f"An error occurred while processing {routes}: {str(e)}")

driver_k.quit()
logging.info("Web scraping completed successfully")

# Convert lists to DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_K = pd.DataFrame(data)

# Convert DataFrame to CSV
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_K.csv"
df_buses_K.to_csv(path, index=False)
logging.info(f"Data saved to CSV: {path}")

print("Successfully Completed")

Successfully Completed


In [ ]:
df_buses_K

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,KSRTC (Kerala) - 794,Super Fast Non AC Seater (2+3),06:40,12:30,05h 50m,INR 269,2 Seats available,3.2\n5,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
1,KSRTC (Kerala) - 2710,Super Fast Non AC Seater (2+3),07:20,13:06,05h 46m,INR 269,33 Seats available,3.0,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
2,KSRTC (Kerala) - 764,Super Fast Non AC Seater (2+3),09:00,14:30,05h 30m,INR 269,38 Seats available,3.0,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
3,KSRTC (Kerala) - 671,Super Fast Non AC Seater (2+3),09:10,14:01,04h 51m,INR 230,30 Seats available,3.4\n6,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
4,KSRTC (Kerala) - 2571,Super Fast Non AC Seater (2+3),09:30,15:55,06h 25m,INR 261,36 Seats available,3.0,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
...,...,...,...,...,...,...,...,...,...,...
90,Sonia Travels,A/C Seater/Sleeper (2+1),23:30,06:15,06h 45m,INR 750,5 Seats available,4.6\n5,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Kottayam
91,Ubc travels,A/C Seater/Sleeper (2+1),01:45,08:00,06h 15m,INR 1700,1 Seat available,3.9\n4,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Kottayam
92,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),02:01,07:55,05h 54m,INR 1400,1 Seat available,3.1\n4,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Kottayam
93,Sonia Travels,A/C Semi Sleeper (2+2),23:45,06:25,06h 40m,INR 700,1 Seat available,4.4,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Kottayam


In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df_1 = pd.read_csv("df_A.csv")
logging.info("CSV file read successfully")

# Initialize the webdriver
driver_A = webdriver.Chrome()

# Initialize lists to store data
Bus_names_A = []
Bus_types_A = []
Start_Time_A = []
End_Time_A = []
Ratings_A = []
Total_Duration_A = []
Prices_A = []
Seats_Available_A = []
Route_names = []
Route_links = []

def scroll_page(driver):
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

# Loop through each row in the DataFrame
for i, r in df_1.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    logging.info(f"Processing route: {routes}")

    try:
        # Navigate to the link
        driver_A.get(link)
        time.sleep(2)

        # Click on elements to reveal bus details
        elements = driver_A.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Click elements to view buses
        try:
            clicks = driver_A.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except NoSuchElementException:
            logging.warning("Button not found, continuing to next route")
            continue
        time.sleep(2)

        # Scroll the page
        scroll_page(driver_A)

        # Extract bus details
        bus_name = driver_A.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_A.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver_A.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver_A.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver_A.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        try:
            rating = driver_A.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        except NoSuchElementException:
            logging.warning("Rating elements not found, skipping")
            continue
        price = driver_A.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver_A.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Bus_names_A.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        Bus_types_A.extend([elem.text for elem in bus_type])
        Start_Time_A.extend([elem.text for elem in start_time])
        End_Time_A.extend([elem.text for elem in end_time])
        Total_Duration_A.extend([elem.text for elem in total_duration])
        Ratings_A.extend([elem.text for elem in rating])
        Prices_A.extend([elem.text for elem in price])
        Seats_Available_A.extend([elem.text for elem in seats])

    except Exception as e:
        logging.error(f"An error occurred while processing {routes}: {str(e)}")

driver_A.quit()
logging.info("Web scraping completed successfully")

# Convert lists to DataFrame
data_1 = {
    'Bus_name': Bus_names_A,
    'Bus_type': Bus_types_A,
    'Start_time': Start_Time_A,
    'End_time': End_Time_A,
    'Total_duration': Total_Duration_A,
    'Price': Prices_A,
    "Seats_Available": Seats_Available_A,
    "Ratings": Ratings_A,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_A = pd.DataFrame(data_1)

# Convert DataFrame to CSV
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_A.csv"
df_buses_A.to_csv(path, index=False)
logging.info(f"Data saved to CSV: {path}")

print("Successfully Completed")

Successfully Completed


In [6]:
df_buses_A

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,APSRTC - 351880,VENNELA (A.C. SLEEPER),00:40,06:05,05h 25m,INR 737,18 Seats available,4.7\n6,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
1,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,07:15,06h 15m,INR 436,26 Seats available,3.8\n31,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
2,APSRTC - 35266,VENNELA (A.C. SLEEPER),01:25,06:49,05h 24m,INR 737,20 Seats available,3.9\n30,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
3,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:45,08:00,06h 15m,INR 436,33 Seats available,4.5\n17,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
4,APSRTC - 3058,VENNELA (A.C. SLEEPER),02:20,07:40,05h 20m,INR 781,18 Seats available,2.9\n9,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
...,...,...,...,...,...,...,...,...,...,...
464,Zingbus Plus,A/C Sleeper (2+1),23:45,06:20,06h 35m,669,25 Seats available,4.7\n175,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam
465,IntrCity SmartBus,AC Sleeper (2+1),23:15,06:40,07h 25m,INR 637,27 Seats available,4.5\n339,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam
466,Svkdt travels,A/C Seater / Sleeper (2+1),22:00,06:00,08h 00m,INR 500,23 Seats available,4.5\n298,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam
467,,,,,,,,,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Visakhapatnam


In [7]:
# read the csv file
df_2=pd.read_csv("df_T.csv")
df_2

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [8]:
# retrive the bus details
driver_T = webdriver.Chrome()
Bus_names_T = []
Bus_types_T = []
Start_Time_T = []
End_Time_T = []
Ratings_T = []
Total_Duration_T = []
Prices_T = []
Seats_Available_T = []
Route_names = []
Route_links = []

for i,r in df_2.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_T.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_T.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_T.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_T.page_source
        
        ActionChains(driver_T).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_T.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_T.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_T.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_T.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_T.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_T.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_T.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_T.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_T.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_T.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_T.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_T.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_T.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_T.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_T.append(ratings.text)
    for price_elem in price:
        Prices_T.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_T.append(seats_elem.text)
print("Successfully Completed")
# from list to convert data frame
data_3 = {
    'Bus_name': Bus_names_T,
    'Bus_type': Bus_types_T,
    'Start_time': Start_Time_T,
    'End_time': End_Time_T,
    'Total_duration': Total_Duration_T,
    'Price': Prices_T,
    "Seats_Available":Seats_Available_T,
    "Ratings":Ratings_T,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_T = pd.DataFrame(data_3)
#convert dataframe to csv
path=r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_T.csv"
df_buses_T.to_csv(path,index=False)

Successfully Completed


In [9]:
df_buses_T

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,FRESHBUS,Electric A/C Seater (2+2),04:35,10:30,05h 55m,530,18 Seats available,4.7\n421,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
1,FRESHBUS,Electric A/C Seater (2+2),05:40,11:45,06h 05m,500,23 Seats available,4.7\n381,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
2,FRESHBUS,Electric A/C Seater (2+2),11:05,17:40,06h 35m,450,32 Seats available,4.6\n344,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
3,AdIntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:05,05:20,06h 15m,INR 378,37 Seats available,4.3\n331,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
4,Sri KVR Travels,A/C Sleeper (2+1),23:35,05:15,05h 40m,INR 560,26 Seats available,4.6\n296,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
...,...,...,...,...,...,...,...,...,...,...
295,VEERA BRAHMENDRA TRAVELS,Non A/C Seater / Sleeper (2+1),21:30,04:20,06h 50m,INR 600,26 Seats available,4.2\n291,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal
296,BRS Travels,NON AC Seater / Sleeper 2+1,22:10,05:00,06h 50m,INR 500,24 Seats available,4.1\n351,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal
297,Indu Travels,Non A/C Seater / Sleeper (2+1),21:45,04:20,06h 35m,INR 500,31 Seats available,4.1\n228,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal
298,BRS Travels,A/C Sleeper (2+1),22:15,05:30,07h 15m,INR 800,18 Seats available,3.9\n337,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal


In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import logging

# Set up logging
logging.basicConfig(filename='scraper_log_G.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df_3 = pd.read_csv("df_G.csv")

# Initialize WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver_G = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver_G, 10)

# Initialize lists to store bus details
Bus_names_G, Bus_types_G, Start_Time_G, End_Time_G = [], [], [], []
Ratings_G, Total_Duration_G, Prices_G, Seats_Available_G = [], [], [], []
Route_names, Route_links = [], []

def safe_find_elements(xpath, timeout=10):
    try:
        return wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    except TimeoutException:
        logging.warning(f"Timeout while finding elements: {xpath}")
        return []

for i, r in df_3.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    logging.info(f"Processing link: {link}")

    try:
        # Loop through each link
        driver_G.get(link)
        time.sleep(2)

        # Click on elements to reveal bus details
        elements = safe_find_elements(f"//a[contains(@href, '{link}')]")
        for element in elements:
            try:
                element.click()
                time.sleep(2)
            except:
                logging.warning(f"Failed to click element for {link}")

        time.sleep(2)
        
        # Scroll the page
        scrolling = True
        scroll_attempts = 0
        while scrolling and scroll_attempts < 5:
            old_page_source = driver_G.page_source
            ActionChains(driver_G).send_keys(Keys.PAGE_DOWN).perform()
            time.sleep(5)
            new_page_source = driver_G.page_source
            if new_page_source == old_page_source:
                scrolling = False
            scroll_attempts += 1

        # Extract bus details
        bus_name = safe_find_elements("//div[@class='travels lh-24 f-bold d-color']")
        bus_type = safe_find_elements("//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = safe_find_elements("//*[@class='dp-time f-19 d-color f-bold']")
        end_time = safe_find_elements("//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = safe_find_elements("//*[@class='dur l-color lh-24']")
        rating = safe_find_elements("//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = safe_find_elements('//*[@class="fare d-block"]')
        seats = safe_find_elements("//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Bus_names_G.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        Bus_types_G.extend([elem.text for elem in bus_type])
        Start_Time_G.extend([elem.text for elem in start_time])
        End_Time_G.extend([elem.text for elem in end_time])
        Total_Duration_G.extend([elem.text for elem in total_duration])
        Ratings_G.extend([elem.text for elem in rating])
        Prices_G.extend([elem.text for elem in price])
        Seats_Available_G.extend([elem.text for elem in seats])

    except Exception as e:
        logging.error(f"Error processing {link}: {str(e)}")
        continue

print("Successfully Completed")

# Convert lists to DataFrame
data_4 = {
    'Bus_name': Bus_names_G,
    'Bus_type': Bus_types_G,
    'Start_time': Start_Time_G,
    'End_time': End_Time_G,
    'Total_duration': Total_Duration_G,
    'Price': Prices_G,
    "Seats_Available": Seats_Available_G,
    "Ratings": Ratings_G,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_G = pd.DataFrame(data_4)

# Convert DataFrame to CSV
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_G.csv"
df_buses_G.to_csv(path, index=False)

# Close the WebDriver
driver_G.quit()

logging.info("Script execution completed")

Successfully Completed


In [11]:
df_buses_G

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,05:50,10h 35m,INR 900,17 Seats available,3.7\n340,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
1,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,07:30,10h 30m,INR 701,12 Seats available,4.6\n595,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
2,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,08:45,10h 15m,INR 660,19 Seats available,4.5\n492,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
3,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:00,08:30,11h 30m,INR 799,8 Seats available,4.4\n636,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
4,Ashray Amrutyog,A/C Sleeper (2+1),22:00,09:30,11h 30m,INR 799,22 Seats available,4.3\n555,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
...,...,...,...,...,...,...,...,...,...,...
110,New Shivbaba Tours And Travels,AC Sleeper (2+1),19:00,03:00,08h 00m,949,29 Seats available,2.7,https://www.redbus.in/bus-tickets/goa-to-sangli,Goa to Sangli
111,New Shivbaba Tours And Travels,AC Sleeper (2+1),19:00,03:00,08h 00m,INR 999,29 Seats available,2.7,https://www.redbus.in/bus-tickets/goa-to-sangli,Goa to Sangli
112,New Shivbaba Tours And Travels,AC Sleeper (2+1),20:30,03:00,06h 30m,INR 999,29 Seats available,2.7,https://www.redbus.in/bus-tickets/goa-to-sangli,Goa to Sangli
113,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),19:00,01:30,06h 30m,INR 1450,25 Seats available,3.8,https://www.redbus.in/bus-tickets/goa-to-sangli,Goa to Sangli


In [12]:
# read the csv file
df_4=pd.read_csv("df_R.csv")
df_4

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import logging

# Set up logging
logging.basicConfig(filename='scraper_log_G.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df = pd.read_csv("df_R.csv")

# Initialize WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

# Initialize lists to store bus details
Bus_names_R = []
Bus_types_R = []
Start_Time_R = []
End_Time_R = []
Ratings_R = []
Total_Duration_R = []
Prices_R = []
Seats_Available_R = []
Route_names_R = []
Route_links_R = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    link = row["Route_link"]
    routes = row["Route_name"]

    try:
        # Navigate to the link
        driver.get(link)
        time.sleep(2)  

        # Click on elements to reveal bus details
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Scroll down the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for i in range(len(bus_name)):
            Bus_names_R.append(bus_name[i].text if i < len(bus_name) else None)
            Bus_types_R.append(bus_type[i].text if i < len(bus_type) else None)
            Start_Time_R.append(start_time[i].text if i < len(start_time) else None)
            End_Time_R.append(end_time[i].text if i < len(end_time) else None)
            Total_Duration_R.append(total_duration[i].text if i < len(total_duration) else None)
            Ratings_R.append(rating[i].text if i < len(rating) else None)
            Prices_R.append(price[i].text if i < len(price) else None)
            Seats_Available_R.append(seats[i].text if i < len(seats) else None)
            Route_links_R.append(link)
            Route_names_R.append(routes)

    except Exception as e:
        logging.error(f"Error processing link {link}: {str(e)}")
        continue

    logging.info(f"Successfully processed link: {link}")

print("Successfully Completed")

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_R,
    'Bus_type': Bus_types_R,
    'Start_time': Start_Time_R,
    'End_time': End_Time_R,
    'Total_duration': Total_Duration_R,
    'Price': Prices_R,
    "Seats_Available": Seats_Available_R,
    "Ratings": Ratings_R,
    'Route_link': Route_links_R,
    'Route_name': Route_names_R
}

df_buses_R = pd.DataFrame(data)

# Save the DataFrame to a CSV file
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_R.csv"
df_buses_R.to_csv(path, index=False)

# Close the WebDriver
driver.quit()

Successfully Completed


In [15]:
df_buses_R

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),17:00,21:30,04h 30m,INR 550,15 Seats available,4.5\n287,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
1,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),17:10,21:55,04h 45m,INR 600,37 Seats available,4.4\n421,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
2,Jain travels regd,A/C Seater / Sleeper (2+1),21:30,04:30,07h 00m,INR 460,27 Seats available,4.1\n251,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
3,AdKalpana Travels,NON AC Seater / Sleeper 2+1,22:00,04:54,06h 54m,INR 300,36 Seats available,3.6\n101,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
4,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),12:50,17:20,04h 30m,INR 550,16 Seats available,4.0\n234,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
...,...,...,...,...,...,...,...,...,...,...
819,Jai Shree Ganesh Travels,A/C Sleeper (2+1),20:39,01:17,04h 38m,INR 1299,36 Seats available,2.8,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)
820,Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,04:30,05h 30m,INR 300,40 Seats available,2.5,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)
821,New shiv mahima travels express & cargo,NON A/C Sleeper (2+1),19:15,00:05,04h 50m,INR 650,30 Seats available,1.8,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)
822,A One Travel Agency,Non A/C Seater / Sleeper (2+1),21:00,02:30,05h 30m,INR 350,36 Seats available,1.6,https://www.redbus.in/bus-tickets/jaipur-to-ko...,Jaipur (Rajasthan) to Kota(Rajasthan)


In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import logging

# Set up logging
logging.basicConfig(filename='scraper_log_SB.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df = pd.read_csv("df_SB.csv")

# Initialize WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

# Initialize lists to store bus details
Bus_names_SB = []
Bus_types_SB = []
Start_Time_SB = []
End_Time_SB = []
Ratings_SB = []
Total_Duration_SB = []
Prices_SB = []
Seats_Available_SB = []
Route_names_SB = []
Route_links_SB = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    link = row["Route_link"]
    routes = row["Route_name"]

    try:
        # Navigate to the link
        driver.get(link)
        time.sleep(2)  

        # Click on elements to reveal bus details
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Scroll down the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for i in range(len(bus_name)):
            Bus_names_SB.append(bus_name[i].text if i < len(bus_name) else None)
            Bus_types_SB.append(bus_type[i].text if i < len(bus_type) else None)
            Start_Time_SB.append(start_time[i].text if i < len(start_time) else None)
            End_Time_SB.append(end_time[i].text if i < len(end_time) else None)
            Total_Duration_SB.append(total_duration[i].text if i < len(total_duration) else None)
            Ratings_SB.append(rating[i].text if i < len(rating) else None)
            Prices_SB.append(price[i].text if i < len(price) else None)
            Seats_Available_SB.append(seats[i].text if i < len(seats) else None)
            Route_links_SB.append(link)
            Route_names_SB.append(routes)

    except Exception as e:
        logging.error(f"Error processing link {link}: {str(e)}")
        continue

    logging.info(f"Successfully processed link: {link}")

print("Successfully Completed")

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_SB,
    'Bus_type': Bus_types_SB,
    'Start_time': Start_Time_SB,
    'End_time': End_Time_SB,
    'Total_duration': Total_Duration_SB,
    'Price': Prices_SB,
    "Seats_Available": Seats_Available_SB,
    "Ratings": Ratings_SB,
    'Route_link': Route_links_SB,
    'Route_name': Route_names_SB
}

df_buses_SB = pd.DataFrame(data)

# Save the DataFrame to a CSV file
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_SB.csv"
df_buses_SB.to_csv(path, index=False)

# Close the WebDriver
driver.quit()

Successfully Completed


In [17]:
df_buses_SB

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),10:55,14:55,04h 00m,INR 486,3 Seats available,3.4\n45,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
1,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),11:00,15:00,04h 00m,INR 486,2 Seats available,3.9\n109,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
2,Greenline,Volvo A/C Seater (2+2),11:30,15:30,04h 00m,INR 486,24 Seats available,4.5\n147,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
3,Greenline (Karunamoyee),Volvo A/C Seater (2+2),12:00,15:50,03h 50m,INR 486,21 Seats available,4.5\n201,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
4,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo B11R Multi Axle Seater (2+2),12:30,16:30,04h 00m,INR 486,15 Seats available,4.3\n253,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
...,...,...,...,...,...,...,...,...,...,...
522,SBSTC-KOLKATA - JHARGRAM - VIA - LODHASULI - 1...,Non AC Seater (2+3),16:20,00:00,07h 40m,INR 82,52 Seats available,4.1\n15,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Ashari
523,SBSTC-KOLKATA - MIDNAPUR - 18:00 (MID DEPOT) -...,Non AC Seater (2+3),18:00,00:00,06h 00m,INR 82,44 Seats available,4.0\n15,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Ashari
524,SBSTC-KOLKATA - MIDNAPUR - 18:45 (MID DEPOT) -...,Non AC Seater (2+3),18:45,00:00,05h 15m,INR 82,35 Seats available,3.5\n13,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Ashari
525,WBTC (CTC) HABRA-DIGHA via Bally - 26|14:00,Non AC Seater (2+3),14:00,18:00,04h 00m,INR 161,46 Seats available,3.9\n5,https://www.redbus.in/bus-tickets/barasat-west...,Barasat (West Bengal) to Digha


In [18]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import logging

# Set up logging
logging.basicConfig(filename='scraper_log_H.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df = pd.read_csv("df_H.csv")

# Initialize WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

# Initialize lists to store bus details
Bus_names_H = []
Bus_types_H = []
Start_Time_H = []
End_Time_H = []
Ratings_H = []
Total_Duration_H = []
Prices_H = []
Seats_Available_H = []
Route_names_H = []
Route_links_H = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    link = row["Route_link"]
    routes = row["Route_name"]

    try:
        # Navigate to the link
        driver.get(link)
        time.sleep(2)  

        # Click on elements to reveal bus details
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Scroll down the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for i in range(len(bus_name)):
            Bus_names_H.append(bus_name[i].text if i < len(bus_name) else None)
            Bus_types_H.append(bus_type[i].text if i < len(bus_type) else None)
            Start_Time_H.append(start_time[i].text if i < len(start_time) else None)
            End_Time_H.append(end_time[i].text if i < len(end_time) else None)
            Total_Duration_H.append(total_duration[i].text if i < len(total_duration) else None)
            Ratings_H.append(rating[i].text if i < len(rating) else None)
            Prices_H.append(price[i].text if i < len(price) else None)
            Seats_Available_H.append(seats[i].text if i < len(seats) else None)
            Route_links_H.append(link)
            Route_names_H.append(routes)

    except Exception as e:
        logging.error(f"Error processing link {link}: {str(e)}")
        continue

    logging.info(f"Successfully processed link: {link}")

print("Successfully Completed")

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_H,
    'Bus_type': Bus_types_H,
    'Start_time': Start_Time_H,
    'End_time': End_Time_H,
    'Total_duration': Total_Duration_H,
    'Price': Prices_H,
    "Seats_Available": Seats_Available_H,
    "Ratings": Ratings_H,
    'Route_link': Route_links_H,
    'Route_name': Route_names_H
}

df_buses_H = pd.DataFrame(data)

# Save the DataFrame to a CSV file
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_H.csv"
df_buses_H.to_csv(path, index=False)

# Close the WebDriver
driver.quit()

Successfully Completed


In [19]:
df_buses_H

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Ram Dalal Holidays,Mercedes Multi-Axle Semi Sleeper (2+2),23:30,07:40,08h 10m,500,25 Seats available,4.5\n592,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,Laxmi holidays,Volvo A/C Semi Sleeper (2+2),23:25,07:45,08h 20m,INR 499,21 Seats available,4.6\n667,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
2,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),23:30,08:00,08h 30m,569,34 Seats available,4.5\n787,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
3,Bedi Travels,Volvo A/C Semi Sleeper (2+2),23:00,08:00,09h 00m,477,25 Seats available,4.8\n801,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
4,Laxmi holidays,Bharat Benz A/C Semi Sleeper (2+2),22:50,07:40,08h 50m,INR 499,42 Seats available,4.2\n313,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...,...
381,Malana Travels,Volvo 9600 Multi Axle Semi-Sleeper (2+2),20:00,06:00,10h 00m,INR 799,28 Seats available,4.5\n394,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi
382,Malana Travels,Volvo A/C Semi Sleeper (2+2),18:00,05:30,11h 30m,INR 709,23 Seats available,4.3\n48,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi
383,Himalayan Nomad Ktc,Volvo A/C Semi Sleeper (2+2),18:00,05:20,11h 20m,656,15 Seats available,4.3\n33,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi
384,AdIntrCity SmartBus,Volvo 9600 A/C Semi Sleeper (2+2),18:15,05:50,11h 35m,INR 1110,23 Seats available,4.6\n330,https://www.redbus.in/bus-tickets/manali-to-delhi,Manali to Delhi


In [20]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import logging

# Set up logging
logging.basicConfig(filename='scraper_log_AS.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df = pd.read_csv("df_AS.csv")

# Initialize WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

# Initialize lists to store bus details
Bus_names_AS = []
Bus_types_AS = []
Start_Time_AS = []
End_Time_AS = []
Ratings_AS = []
Total_Duration_AS = []
Prices_AS = []
Seats_Available_AS = []
Route_names_AS = []
Route_links_AS = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    link = row["Route_link"]
    routes = row["Route_name"]

    try:
        # Navigate to the link
        driver.get(link)
        time.sleep(2)  

        # Click on elements to reveal bus details
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Scroll down the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for i in range(len(bus_name)):
            Bus_names_AS.append(bus_name[i].text if i < len(bus_name) else None)
            Bus_types_AS.append(bus_type[i].text if i < len(bus_type) else None)
            Start_Time_AS.append(start_time[i].text if i < len(start_time) else None)
            End_Time_AS.append(end_time[i].text if i < len(end_time) else None)
            Total_Duration_AS.append(total_duration[i].text if i < len(total_duration) else None)
            Ratings_AS.append(rating[i].text if i < len(rating) else None)
            Prices_AS.append(price[i].text if i < len(price) else None)
            Seats_Available_AS.append(seats[i].text if i < len(seats) else None)
            Route_links_AS.append(link)
            Route_names_AS.append(routes)

    except Exception as e:
        logging.error(f"Error processing link {link}: {str(e)}")
        continue

    logging.info(f"Successfully processed link: {link}")

print("Successfully Completed")

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_AS,
    'Bus_type': Bus_types_AS,
    'Start_time': Start_Time_AS,
    'End_time': End_Time_AS,
    'Total_duration': Total_Duration_AS,
    'Price': Prices_AS,
    "Seats_Available": Seats_Available_AS,
    "Ratings": Ratings_AS,
    'Route_link': Route_links_AS,
    'Route_name': Route_names_AS
}

df_buses_AS = pd.DataFrame(data)

# Save the DataFrame to a CSV file
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_AS.csv"
df_buses_AS.to_csv(path, index=False)

# Close the WebDriver
driver.quit()

Successfully Completed


In [21]:
df_buses_AS

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Assam State Transport Corporation (ASTC) - 139720,Volvo AC Seater 2+2,13:45,18:00,04h 15m,INR 298,23 Seats available,4.2\n42,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
1,Orient Transline,A/C Seater (2+1),11:30,15:50,04h 20m,INR 380.95,10 Seats available,3.8\n314,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
2,Orient Transline,A/C Seater (2+1),13:00,17:10,04h 10m,INR 380.95,15 Seats available,4.2\n163,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
3,Mahi Travels(Under ASTC),A/C Seater (2+2),14:15,17:50,03h 35m,389,18 Seats available,4.5\n140,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
4,Mahalaxmi Travels,A/C Seater (2+2),14:35,18:15,03h 40m,INR 300,33 Seats available,3.8\n24,https://www.redbus.in/bus-tickets/guwahati-to-...,Guwahati to Tezpur
...,...,...,...,...,...,...,...,...,...,...
201,SHIVAM TRAVELS,Bharat Benz A/C Seater (2+1),12:15,17:00,04h 45m,INR 550,4 Seats available,4.0,https://www.redbus.in/bus-tickets/bokakhat-to-...,Bokakhat to Dibrugarh
202,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,12:30,17:15,04h 45m,INR 735.16,9 Seats available,4.5\n12,https://www.redbus.in/bus-tickets/bokakhat-to-...,Bokakhat to Dibrugarh
203,Bashudev Transline (Under ASTC),A/C Seater (2+1),12:35,18:00,05h 25m,INR 550,8 Seats available,3.9\n8,https://www.redbus.in/bus-tickets/bokakhat-to-...,Bokakhat to Dibrugarh
204,Aniruddha Travels,NON A/C Seater/ Sleeper (2+1),23:15,04:00,04h 45m,INR 550,42 Seats available,3.1\n5,https://www.redbus.in/bus-tickets/bokakhat-to-...,Bokakhat to Dibrugarh


In [22]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import logging

# Set up logging
logging.basicConfig(filename='scraper_log_UP.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df = pd.read_csv("df_UP.csv")

# Initialize WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

# Initialize lists to store bus details
Bus_names_UP = []
Bus_types_UP = []
Start_Time_UP = []
End_Time_UP = []
Ratings_UP = []
Total_Duration_UP = []
Prices_UP = []
Seats_Available_UP = []
Route_names_UP = []
Route_links_UP = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    link = row["Route_link"]
    routes = row["Route_name"]

    try:
        # Navigate to the link
        driver.get(link)
        time.sleep(2)  

        # Click on elements to reveal bus details
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Scroll down the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for i in range(len(bus_name)):
            Bus_names_UP.append(bus_name[i].text if i < len(bus_name) else None)
            Bus_types_UP.append(bus_type[i].text if i < len(bus_type) else None)
            Start_Time_UP.append(start_time[i].text if i < len(start_time) else None)
            End_Time_UP.append(end_time[i].text if i < len(end_time) else None)
            Total_Duration_UP.append(total_duration[i].text if i < len(total_duration) else None)
            Ratings_UP.append(rating[i].text if i < len(rating) else None)
            Prices_UP.append(price[i].text if i < len(price) else None)
            Seats_Available_UP.append(seats[i].text if i < len(seats) else None)
            Route_links_UP.append(link)
            Route_names_UP.append(routes)

    except Exception as e:
        logging.error(f"Error processing link {link}: {str(e)}")
        continue

    logging.info(f"Successfully processed link: {link}")

print("Successfully Completed")

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_UP,
    'Bus_type': Bus_types_UP,
    'Start_time': Start_Time_UP,
    'End_time': End_Time_UP,
    'Total_duration': Total_Duration_UP,
    'Price': Prices_UP,
    "Seats_Available": Seats_Available_UP,
    "Ratings": Ratings_UP,
    'Route_link': Route_links_UP,
    'Route_name': Route_names_UP
}

df_buses_UP = pd.DataFrame(data)

# Save the DataFrame to a CSV file
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_UP.csv"
df_buses_UP.to_csv(path, index=False)

# Close the WebDriver
driver.quit()

Successfully Completed


In [23]:
df_buses_UP

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,FlixBus,AC Seater (2+2),23:35,05:00,05h 25m,INR 209.44,31 Seats available,4.6\n34,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
1,New Khaira Transport,A/C Seater / Sleeper (2+1),20:40,03:30,06h 50m,630,2 Seats available,3.1\n11,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
2,Rahul Travels,AshokLeyland Stile A/C,21:00,02:00,05h 00m,629,27 Seats available,1.9\n6,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
3,FlixBus,Bharat Benz A/C Seater (2+2),17:15,22:25,05h 10m,INR 250.8,37 Seats available,5.0\n9,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
4,SHRI RAM TRAVELS,A/C Seater (2+2),23:59,05:15,05h 16m,INR 740,18 Seats available,3.9\n22,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
...,...,...,...,...,...,...,...,...,...,...
211,Abhay Travels (I.R.T.C),Non A/C Seater / Sleeper (2+1),20:30,06:00,09h 30m,INR 1300,30 Seats available,New,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
212,Yuvraj Travels,NON AC Seater / Sleeper 2+1,20:00,06:00,10h 00m,INR 550,40 Seats available,4.1,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
213,Yuvraj Travels,NON AC Seater / Sleeper 2+1,21:30,06:00,08h 30m,INR 550,40 Seats available,4.1,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)
214,Shree Krishna Shtabadi Travels,NON A/C Sleeper (2+1),21:00,06:30,09h 30m,INR 1200,32 Seats available,1.4,https://www.redbus.in/bus-tickets/delhi-to-far...,Delhi to Farrukhabad (Uttar Pradesh)


In [26]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import logging

# Set up logging
logging.basicConfig(filename='scraper_log_WB.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Read the CSV file
df = pd.read_csv("df_WB.csv")

# Initialize WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

# Initialize lists to store bus details
Bus_names_WB = []
Bus_types_WB = []
Start_Time_WB = []
End_Time_WB = []
Ratings_WB = []
Total_Duration_WB = []
Prices_WB = []
Seats_Available_WB = []
Route_names_WB = []
Route_links_WB = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    link = row["Route_link"]
    routes = row["Route_name"]

    try:
        # Navigate to the link
        driver.get(link)
        time.sleep(2)  

        # Click on elements to reveal bus details
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # Scroll down the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for i in range(len(bus_name)):
            Bus_names_WB.append(bus_name[i].text if i < len(bus_name) else None)
            Bus_types_WB.append(bus_type[i].text if i < len(bus_type) else None)
            Start_Time_WB.append(start_time[i].text if i < len(start_time) else None)
            End_Time_WB.append(end_time[i].text if i < len(end_time) else None)
            Total_Duration_WB.append(total_duration[i].text if i < len(total_duration) else None)
            Ratings_WB.append(rating[i].text if i < len(rating) else None)
            Prices_WB.append(price[i].text if i < len(price) else None)
            Seats_Available_WB.append(seats[i].text if i < len(seats) else None)
            Route_links_WB.append(link)
            Route_names_WB.append(routes)

    except Exception as e:
        logging.error(f"Error processing link {link}: {str(e)}")
        continue

    logging.info(f"Successfully processed link: {link}")

print("Successfully Completed")

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_WB,
    'Bus_type': Bus_types_WB,
    'Start_time': Start_Time_WB,
    'End_time': End_Time_WB,
    'Total_duration': Total_Duration_WB,
    'Price': Prices_WB,
    "Seats_Available": Seats_Available_WB,
    "Ratings": Ratings_WB,
    'Route_link': Route_links_WB,
    'Route_name': Route_names_WB
}

df_buses_WB = pd.DataFrame(data)

# Save the DataFrame to a CSV file
path = r"C:\Users\SURYA\Desktop\Project2\Bus_Details\df_buses_WB.csv"
df_buses_WB.to_csv(path, index=False)

# Close the WebDriver
driver.quit()

Successfully Completed


In [27]:
df_buses_WB

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Rudraksha City Service,A/C Seater (2+3),13:00,18:26,05h 26m,INR 370,8 Seats available,1.9\n15,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
1,Aradhana Bus Service,Non AC Seater (2+3),16:50,23:30,06h 40m,INR 350,54 Seats available,2.1,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
2,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,23:55,05h 20m,463,20 Seats available,1.1\n10,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
3,Satya Paribahan,A/C Seater (2+3),23:00,05:10,06h 10m,INR 370,20 Seats available,2.8\n32,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
4,Snemita Paribahan (Shinjini),AC Seater (2+3),23:35,05:00,05h 25m,INR 370,29 Seats available,2.4\n17,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
...,...,...,...,...,...,...,...,...,...,...
272,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,16:10,03h 25m,INR 108,44 Seats available,2.9,https://www.redbus.in/bus-tickets/habra-to-nan...,Habra to Nandakumar (west bengal)
273,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,15:40,02h 55m,INR 87,44 Seats available,4.5,https://www.redbus.in/bus-tickets/habra-to-kol...,Habra to Kolaghat
274,WBTC (CTC) KOLKATA - MAYAPUR - 47|15:30,Non AC Seater (2+3),15:30,20:30,05h 00m,INR 108,53 Seats available,3.4\n5,https://www.redbus.in/bus-tickets/kolkata-to-m...,Kolkata to Mayapur ISKCON
275,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,16:40,03h 55m,INR 130,46 Seats available,4.2,https://www.redbus.in/bus-tickets/habra-to-heria,Habra to Heria


In [28]:
import pandas as pd
import mysql.connector
import numpy as np

In [31]:
# csv to dataframe
df_buses_1=pd.read_csv("df_buses_K.csv")
df_buses_2=pd.read_csv("df_buses_A.csv")
df_buses_3=pd.read_csv("df_buses_T.csv")
df_buses_4=pd.read_csv("df_buses_G.csv")
df_buses_5=pd.read_csv("df_buses_R.csv")
df_buses_6=pd.read_csv("df_buses_SB.csv")
df_buses_7=pd.read_csv("df_buses_H.csv")
df_buses_8=pd.read_csv("df_buses_AS.csv")
df_buses_9=pd.read_csv("df_buses_UP.csv")
df_buses_10=pd.read_csv("df_buses_WB.csv")

Final_df=pd.concat([df_buses_K,df_buses_A,df_buses_T,df_buses_G,df_buses_R,df_buses_SB,
                    df_buses_H,df_buses_AS,df_buses_UP,df_buses_WB],ignore_index=True)
Final_df

FileNotFoundError: [Errno 2] No such file or directory: 'df_buses_K.csv'